# The Battle of Neighborhoods


### Snto André vs Uberlândia


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.2.1 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
brneigh = pd.read_csv('https://www.dropbox.com/s/wklo4s23x2xjdyn/tabela202.csv?dl=1')
brneigh.shape


(14227, 3)

In [3]:
# Delete columm Situação do domicílio 
brneigh.drop(['Situação do domicílio'], axis=1, inplace=True)
brneigh.shape
brneigh.head()


,Bairro,Total
0,Centro - Alta Floresta D'Oeste (RO),1960
1,Liberdade - Alta Floresta D'Oeste (RO),1075
2,Cidade Alta - Alta Floresta D'Oeste (RO),1175
3,Santa Felicidade - Alta Floresta D'Oeste (RO),2833
4,Princesa Isabel - Alta Floresta D'Oeste (RO),3067


In [4]:
# Function to get the geograpical coordinate
def getGeoC(neighborhood, city):
    
    address = neighborhood + ' ' + city
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    return latitude, longitude

In [5]:
# Delete rows different of Santo André and Uberlândia
# Increase Population: Santo André + 5,037%, Uberlândia + 15,74%
# Create columm income: Santo André 26035.49, Uberlândia 48585.36
try:
    del map_uber
    del map_sta 
except:
    pass
income_rp = 26035.49
income_udi = 48585.36
data_uber = pd.DataFrame(columns=['City','Neighborhood','Population','Income','Latitude','Longitude'])
data_sta = pd.DataFrame(columns=['City','Neighborhood','Population','Income','Latitude','Longitude'])
for x in range(len(brneigh.index)): 
    try:
        if (brneigh['Bairro'].loc[x].find('Santo André (SP)') != -1):
            sep = brneigh['Bairro'].loc[x].split(' - ')
            my_latitude, my_longitude = getGeoC(sep[0], sep[1])
            data_sta = data_sta.append({"City": sep[1], "Neighborhood": sep[0], "Population": int(float(brneigh['Total'].loc[x]) * 1.0537) , \
                                  "Income": income_rp, "Latitude": my_latitude, "Longitude": my_longitude}, ignore_index=True)
        else:
            if (brneigh['Bairro'].loc[x].find('Uberlândia (MG)') != -1):
                sep = brneigh['Bairro'].loc[x].split(' - ')
                my_latitude, my_longitude = getGeoC(sep[0], sep[1])
                data_uber  = data_uber.append({"City": sep[1], "Neighborhood": sep[0], "Population": int(float(brneigh['Total'].loc[x]) * 1.1574), \
                                      "Income": income_udi, "Latitude": my_latitude, "Longitude": my_longitude}, ignore_index=True) 
            else:
                pass
    except:
        pass
else:
    pass
data_sta.shape
data_uber.shape

(51, 6)

In [6]:
data_sta.head()

,City,Neighborhood,Population,Income,Latitude,Longitude
0,Santo André (SP),Vila Metalúrgica,11552,26035.49,-23.620442,-46.539045
1,Santo André (SP),Vila Camilópolis,17631,26035.49,-23.621858,-46.527778
2,Santo André (SP),Jardim Utinga,4823,26035.49,-23.618624,-46.520350
3,Santo André (SP),Jardim das Maravilhas,5051,26035.49,-23.622644,-46.516871
4,Santo André (SP),Vila Lucinda,6453,26035.49,-23.628069,-46.514714


In [7]:
data_uber.head()

,City,Neighborhood,Population,Income,Latitude,Longitude
0,Uberlândia (MG),Tibery,21563,48585.36,-18.902763,-48.249193
1,Uberlândia (MG),Brasil,14700,48585.36,-18.992151,-49.445548
2,Uberlândia (MG),Marta Helena,11297,48585.36,-18.981524,-49.480686
3,Uberlândia (MG),Bom Jesus,5168,48585.36,-18.901936,-48.273286
4,Uberlândia (MG),Umuarama,4324,48585.36,-18.881731,-48.257594


#### Let's visualize the neighborhoods in Santo André

In [13]:
# create map of Santo André using latitude and longitude values
latitude, longitude = getGeoC('Santo André', 'SP')
map_sta = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(data_sta['Latitude'], data_sta['Longitude'], data_sta['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sta)  
    
map_sta


#### Let's visualize the neighborhoods in Uberlândia

In [14]:
# create map of Uberlândia using latitude and longitude values
lat_uber, long_uber = getGeoC('Uberlândia', 'MG')
map_uber = folium.Map(location=[lat_uber, long_uber], zoom_start=12)

# add markers to map
for lat, lng, label in zip(data_uber['Latitude'], data_uber['Longitude'], data_uber['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_uber)  
    
map_uber


#### Define Foursquare Credentials and Version

In [15]:
# @hidden_cell  
CLIENT_ID = 'C5LN4CIJKKDIULV0LXZ50GZDDAIS1DDZOWUM2HBN5KULXE2B' # your Foursquare ID
CLIENT_SECRET = 'SE3SXQYY5523JIJ4YDLC1QJXEUMWNJ032LJAW5GJCCISPOJU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500


#### Let's create a function to inform all venues

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
sta_venues = getNearbyVenues(names=data_sta['Neighborhood'], latitudes=data_sta['Latitude'], \
                                 longitudes=data_sta['Longitude'])

Vila Metalúrgica
Vila Camilópolis
Jardim Utinga
Jardim das Maravilhas
Vila Lucinda
Jardim Santo Antônio
Vila Francisco Matarazzo
Parque Oratório
Parque das Nações
Santa Terezinha
Campestre
Bangú
Jardim
Santa Maria
Vila Palmares
Vila Sacadura Cabral
Vila Alpina
Vila Guiomar
Centro
Casa Branca
Parque Marajoara
Vila Homero Thon
Vila América
Bairro Silveira
Vila Alzira
Vila Assunção
Vila Bastos
Jardim Bela Vista
Vila Alice
Vila Príncipe de Gales
Vila Valparaíso
Vila Floresta
Vila Gilda
Paraíso
Vila Pires
Vila Humaitá
Vila Guarani
Jardim Marek
Cidade São Jorge
Vila Progresso
Vila Helena
Vila Scarpelli
Jardim Bom Pastor
Jardim Stella
Jardim Cristiane
Vila Linda
Vila Junqueira
Jardim Ipanema
Vila Guaraciaba
Condomínio Maracanã
Vila Tibiriçá
Vila Suíça
Vila Lutécia
Jardim Santa Cristina
Jardim do Estádio
Jardim Alvorada
Sítio dos Vianas
Vila Luzita
Jardim Santo André
Jardim Irene
Jardim João Ramalho
Cata Preta
Parque do Pedroso
Recreio da Borda do Campo
Parque Miami
Jardim Riviera
Fazenda dos 

In [34]:
uber_venues = getNearbyVenues(names=data_uber['Neighborhood'], latitudes=data_uber['Latitude'], \
                                 longitudes=data_uber['Longitude'])

Tibery
Brasil
Marta Helena
Bom Jesus
Umuarama
Jardim Patrícia
Presidente Roosevelt
Guarani
Mansour
Luizote de Freitas
Maravilha
Tubalina
Martins
Osvaldo Rezende
Planalto
Minas Gerais
Santa Mônica
Jardim Brasília
Nossa Senhora Aparecida
São José
Patrimônio
Cazeca
Segismundo Pereira
Cidade Jardim
Morada do Sol
Chácaras Tubalina e Quartel
Tocantins
Morada da Colina
Residencial Gramado
Jardim Karaíba
Daniel Fonseca
Saraiva
São Jorge
Tabajaras
Centro
Taiaman
Jardim das Palmeiras
Vigilato Pereira
Laranjeiras
Fundinho
Morumbi
Dona Zulmira
Santa Luzia
Lídice
Pacaembú
Jaraguá
Jardim Canaã
Jardim Holanda
Custódio Pereira
Panorama
Granada


In [41]:
print('Total Venues Santo André: ' , sta_venues.shape)
print('Total Venues Uberlândia: ' , uber_venues.shape)

indexsta_ph = sta_venues[sta_venues['Venue Category'] != 'Pharmacy'].index
sta_venues.drop(indexsta_ph , inplace=True)
print('Pharmacy Venues Santo André: ', sta_venues.shape)

indexuber_ph = uber_venues[uber_venues['Venue Category'] != 'Pharmacy'].index
uber_venues.drop(indexuber_ph , inplace=True)
print('Pharmacy Venues Uberlândia: ', uber_venues.shape)


Total Venues Santo André:  (28, 7)
Total Venues Uberlândia:  (12, 7)
Pharmacy Venues Santo André:  (28, 7)
Pharmacy Venues Uberlândia:  (12, 7)


In [42]:
# Group rows by neighborhood
sta_grouped = sta_venues.groupby('Neighborhood').count().reset_index()
sta_grouped.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis = 1, \
                 inplace=True)

uber_grouped = uber_venues.groupby('Neighborhood').count().reset_index()
uber_grouped.drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Latitude','Venue Longitude','Venue Category'], axis = 1, \
                 inplace=True)

### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [20]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 4, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Bakery,Italian Restaurant,Gastropub,Restaurant,Cocktail Bar,Farmers Market,Pizza Place,American Restaurant
1,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Health Food Store,Pub,Wine Bar,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
2,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Clothing Store,Jazz Club
3,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Portuguese Restaurant,Ramen Restaurant,Poke Place
4,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Coffee Shop,Restaurant,Italian Restaurant,Candy Store,Nightclub,Baby Store,Dog Run


Let's visualize the resulting clusters

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Summerhill West,Coffee Shop,Light Rail Station,Supermarket,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant
61,Rathnelly,Coffee Shop,Light Rail Station,Supermarket,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant
62,South Hill,Coffee Shop,Light Rail Station,Supermarket,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant
63,Forest Hill SE,Coffee Shop,Light Rail Station,Supermarket,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant
64,Deer Park,Coffee Shop,Light Rail Station,Supermarket,Fried Chicken Joint,Liquor Store,Restaurant,Bank,Bagel Shop,Pub,Sushi Restaurant


#### Cluster 2

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. James Town,Coffee Shop,Café,Bakery,Italian Restaurant,Gastropub,Restaurant,Cocktail Bar,Farmers Market,Pizza Place,American Restaurant
1,The Beaches,Trail,Health Food Store,Pub,Wine Bar,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
2,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Farmers Market,Seafood Restaurant,Restaurant,Cheese Shop,Clothing Store,Jazz Club
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Burger Joint,Salad Place,Bubble Tea Shop,Portuguese Restaurant,Ramen Restaurant,Poke Place
4,Christie,Grocery Store,Café,Park,Coffee Shop,Restaurant,Italian Restaurant,Candy Store,Nightclub,Baby Store,Dog Run
5,Studio District,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Yoga Studio,Cheese Shop,Fish Market,Italian Restaurant
6,Lawrence Park,Clothing Store,Park,Coffee Shop,Grocery Store,Gift Shop,Seafood Restaurant,Salon / Barbershop,Restaurant,Bus Line,Café
8,Davisville North,Park,Food & Drink Shop,Dance Studio,Hotel,Dog Run,Sandwich Place,Breakfast Spot,Gym / Fitness Center,Department Store,Donut Shop
9,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Café,Coffee Shop,Gym,Sushi Restaurant,Farmers Market,Diner
11,Stn A PO Boxes,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Beer Bar,Hotel,Café,Gym


#### Cluster 3

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Roselawn,Garden,Wine Bar,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,Park,Playground,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
40,Forest Hill North & West,Park,Trail,Jewelry Store,Sushi Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
41,Forest Hill Road Park,Park,Trail,Jewelry Store,Sushi Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
55,Moore Park,Gym,Park,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
56,Summerhill East,Gym,Park,Trail,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 5

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,CN Tower,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
66,King and Spadina,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
67,Railway Lands,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
68,Harbourfront West,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
69,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
70,South Niagara,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
71,Island airport,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop
